In [ ]:
import pandas as pd
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

import backtrader as bt
import backtrader.analyzers as btanalyzers
import matplotlib
from datetime import datetime
import pandas as pd
from pathlib import Path
from datetime import datetime

In [ ]:
stock_data = PyTickerSymbols()
sp100_yahoo = stock_data.get_sp_100_nyc_yahoo_tickers()
sp100_yahoo

## Select the Stock Universe
For the moment just use a few tickers for testing.
Then scale up to 100 (nasdaq100 or sp100)
Then eventually Russel 3000

In [ ]:
stocks = ['AAPL', 'MMM', 'AMZN', 'CLX']
data_path  = Path('/Volumes/extdisk/yfinance/prices')
#data_path  = Path('/Volumes/extdisk/fin_sentiment_dataset/full_price_hist')
from_date=datetime(2017, 1, 3),
to_date=datetime(2022, 12, 31),

In [ ]:
# need to instatiate cerebro before we add stocks
cerebro = bt.Cerebro()

# load the stock data
stocks = ['AAPL', 'MSFT', 'AMZN', 'TSLA']
for s in stocks: 
    #data = bt.feeds.YahooFinanceData(dataname = s, fromdate = datetime(2010, 1, 1), todate = datetime(2020, 1, 1))
    fname = s + '.csv'
    datapath = data_path/fname
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        fromdate=datetime(2017, 1, 1),
        todate=datetime(2022, 12, 31),
        reverse=False)
    cerebro.adddata(data, name = s)

In [ ]:
class MomentumStrategy(bt.Strategy):
    
    params = (
        ('fast_length', 5),
        ('slow_length', 25)
    )
     
    def __init__(self):
        self.crossovers = []
         
        for d in self.datas: 
            ma_fast = bt.ind.SMA(d, period = self.params.fast_length)
            ma_slow = bt.ind.SMA(d, period = self.params.slow_length)
 
            self.crossovers.append(bt.ind.CrossOver(ma_fast, ma_slow))
 
    def next(self):
        for i, d in enumerate(self.datas):
            if not self.getposition(d).size:
                if self.crossovers[i] > 0: 
                    self.buy(data = d)
            elif self.crossovers[i] < 0: 
                self.close(data = d)

In [ ]:
cerebro.addstrategy(MomentumStrategy)
cerebro.broker.setcash(10000.0)
cerebro.addsizer(bt.sizers.PercentSizer, percents = 25)

In [ ]:
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name = "sharpe")
cerebro.addanalyzer(btanalyzers.Returns,     _name = "returns")
cerebro.addanalyzer(btanalyzers.Transactions, _name = "trans")

In [ ]:
back = cerebro.run()

In [ ]:
cerebro.broker.getvalue()

In [ ]:
back[0].analyzers.returns.get_analysis()['rnorm100']

In [ ]:
back[0].analyzers.sharpe.get_analysis()

In [ ]:
back[0].analyzers.returns.print()

In [ ]:
%matplotlib inline
cerebro.plot(iplot=False)